# Recurrent neural network(RNN) 


Recurrent neural network(RNN)은 Text, 음성, 시계열 데이터등 순차적인(Sequential) 데이터들에 대한 분석을 할때 주로 사용하는 알고리즘입니다. <br>
본 문서에서는 지난시간에 배웠던 RNN을 활용해 '임의의 순서'를 갖는 문자열 데이터로부터<br> 'I LOVE YOU'라는 순서를 갖는 문자열을 '나는 너를 사랑해'로 출력하는 RNN을 학습시켜 보겠습니다.<br>

본 예제에서는<br>
Input: ILOVEYOU -> Output: 나는너를사랑해 <br>
되도록 학습 시킬 것 입니다.<br><br>
이렇게 input과 output이 모두 Sequence인 모델을 <br>
- sequence-to-sequence(seq2seq) 
- many-to-many model
- encoder, decoder model

이라고 부릅니다. 대표적인 예로는 Neural Machine Translation(NMT)가 있습니다.

In [1]:
#-*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint #데이터를 보기 좋게 출력해주는 모듈

자연어 처리에서는 보통 데이터를 다룰때 <br>
1.단어사전) '텍스트데이터':'인덱스' <br>
2.인덱스를 단어로변환) '인덱스':'텍스트데이터'<br>
의 형태로 구축해서 사용합니다. <br>
데이터를 처리할때 '텍스트데이터' 형태로 그대로 쓰기보다 '인덱스'를 사용해서 처리하는 것이 통상적이므로 여기서도 같은 방법을 사용하겠습니다.

In [2]:
text_data_en = ['I','L','O','V','E','Y','O','U']

# 단어사전 구축
vocab_en = {}
index_en = 0
for text in text_data_en:
    if text not in vocab_en:
        vocab_en[text] = index_en
        index_en = index_en + 1

# 인덱스->단어 사전 구축
index2Char_en = {}
for text, index in vocab_en.items():
    index2Char_en[index] = text

print(vocab_en)
print(index2Char_en)

# one hot representation 차원 계산
one_hot_dimension_en = len(vocab_en)
print("one_hot_dimension_en:", one_hot_dimension_en)

# 넣어줄 데이터를 np.float32 타입으로 변환해줍니다. (지정하지 않아서 타입이 안맞을 경우 에러 발생)
# 각 문자를 벡터로 나타내기 위해 one-hot representation 형태를 사용합니다.
one_hot_embedding_matrix_en = np.array([[1, 0, 0, 0, 0, 0, 0],     # I
                                       [0, 1, 0, 0, 0, 0, 0],  # L
                                       [0, 0, 1, 0, 0, 0, 0],  # O
                                       [0, 0, 0, 1, 0, 0, 0],  # V
                                       [0, 0, 0, 0, 1, 0, 0],  # E
                                       [0, 0, 0, 0, 0, 1, 0],  # Y
                                       [0, 0, 0, 0, 0, 0, 1]], dtype=np.float32) # U

print(one_hot_embedding_matrix_en)

{'Y': 5, 'V': 3, 'O': 2, 'I': 0, 'L': 1, 'U': 6, 'E': 4}
{0: 'I', 1: 'L', 2: 'O', 3: 'V', 4: 'E', 5: 'Y', 6: 'U'}
one_hot_dimension_en: 7
[[ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]]


In [3]:
text_data_ko = ['나','는','너','를','사','랑','해', '<eos>']


# 단어사전 구축
vocab_ko = {}
index_ko = 0
for text in text_data_ko:
    if text not in vocab_ko:
        vocab_ko[text] = index_ko
        index_ko = index_ko + 1

# 인덱스->단어 사전 구축
index2Char_ko = {}
for text, index in vocab_ko.items():
    index2Char_ko[index] = text

print(vocab_ko)
print(index2Char_ko)

# one hot representation 차원 계산
one_hot_dimension_ko = len(vocab_en)
print("one_hot_dimension_ko:", one_hot_dimension_ko)

{'랑': 5, '해': 6, '는': 1, '사': 4, '나': 0, '를': 3, '<eos>': 7, '너': 2}
{0: '나', 1: '는', 2: '너', 3: '를', 4: '사', 5: '랑', 6: '해', 7: '<eos>'}
one_hot_dimension_ko: 7


In [4]:
y_data = [[0, 1, 2, 3, 4, 5, 6, 7]]    # Target은 '나는너를사랑해<eos>' 입니다.
y_string = [index2Char_ko[y] for y in y_data[0]]
print("y_data", y_data)
print("y_string", y_string)

# 학습에 사용할 "IILOVEYOU"를 입력합니다.
x_input_char = ['I','L','O','V','E','Y','O','U']

# 문자열을 index로 바꾸겠습니다.
x_input_index = []

for x in x_input_char:    
    x_input_index.append(vocab_en[x])  #  (문자->index) 로 변환

print("x_input_index", x_input_index)    
x_input_string = [index2Char_en[x] for x in x_input_index]
print("x_input_string", x_input_string)

# index를 이용하여 one_hot_vector 모양으로 바꾸겠습니다.
x_input_one_hot = []
for x in x_input_index:
    x_input_one_hot.append(one_hot_embedding_matrix_en[x])
x_input_one_hot = np.array([x_input_one_hot]) #TensorFlow에서는 첫차원이 Batch size이므로 차원을 한칸 밀어주기 위해 []를 추가합니다.
print("x_input_one_hot")
print(x_input_one_hot) # Batch, 문장내 문자수, 문자 차원수


y_data [[0, 1, 2, 3, 4, 5, 6, 7]]
y_string ['나', '는', '너', '를', '사', '랑', '해', '<eos>']
x_input_index [0, 1, 2, 3, 4, 5, 2, 6]
x_input_string ['I', 'L', 'O', 'V', 'E', 'Y', 'O', 'U']
x_input_one_hot
[[[ 1.  0.  0.  0.  0.  0.  0.]
  [ 0.  1.  0.  0.  0.  0.  0.]
  [ 0.  0.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  1.  0.  0.]
  [ 0.  0.  0.  0.  0.  1.  0.]
  [ 0.  0.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  1.]]]


In [5]:
# parameters

sequence_length = 8 #len(x_input_char)
input_dimension = 7 #one_hot_dimension
num_classes = 8 # 문자수
hidden_size = 2 # RNN의 hidden layer 차원
num_layers = 2 # Multi layer RNN의 layer 개수
batch_size = 1 # 현재는 문장 1개로 하기 때문에 1
learning_rate = 0.01 # Adam optimizer의 Learning rate

In [6]:
X_input = tf.placeholder(tf.float32, [None, sequence_length, input_dimension]) #Padding까지 고려된 크기
Y = tf.placeholder(tf.int32, [None, sequence_length]) # RNN의 output은 character에 대한 index (Class 혹은 Label)
seq_length = tf.placeholder(tf.int32) #나중에 가변적인 값을 넣을 때 사용 ex) [['really','good'],['good', 'pad']] 의 경우 Seq_length는 [2,1]

In [7]:
sess = tf.InteractiveSession()

### Multi-Bidirectional RNN
지난 시간에 배운 Multi-Bidirectional RNN을 적용해보겠습니다

In [8]:
def Multi_Bi_RNN(x_input, sequence_length, hidden_size, num_layers, rnn_type='rnn'):
    with tf.variable_scope('multi_bidirectional_rnn') as scope:
        
        
        cell_fw = None
        cell_bw = None
        if(rnn_type == 'rnn'):
            cell_fw = [tf.contrib.rnn.BasicRNNCell(num_units=hidden_size) for i in range(num_layers)]
            cell_bw = [tf.contrib.rnn.BasicRNNCell(num_units=hidden_size) for i in range(num_layers)]
            
        elif(rnn_type == 'lstm'):
            cell_fw = [tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True) for i in range(num_layers)]
            cell_bw = [tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True) for i in range(num_layers)]
            #state_is_tuple = True로 하면 Cell State와 Hidden State의 값을 tuple형태로 분리해서 보여줍니다.
        cell_fw = tf.contrib.rnn.MultiRNNCell(cells=cell_fw, state_is_tuple=True)
        cell_bw = tf.contrib.rnn.MultiRNNCell(cells=cell_bw, state_is_tuple=True)
            
        outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_input, 
                                                          sequence_length=sequence_length,
                                                          dtype=tf.float32)
        return outputs, states

In [9]:
outputs, states = Multi_Bi_RNN(X_input, seq_length, hidden_size, num_layers, rnn_type='lstm')

### Fully Connected Layer
RNN에서 나온 값들을 Fully Connected Layer의 Input 값으로 사용합니다.

In [10]:
# x_for_fc_0 = tf.reshape(outputs[0], [-1, hidden_size])
# x_for_fc_1 = tf.reshape(outputs[1], [-1, hidden_size])
# x_for_fc = tf.concat(x_for_fc_0, x_for_fc_1, axis = 1)

x_for_fc = tf.reshape(outputs, [-1, hidden_size * 2]) # Bidirectional 이기 때문에 Output이 2개 나오기 때문에 hidden_size * 2

outputs = tf.contrib.layers.fully_connected(
    inputs=x_for_fc, num_outputs=num_classes, activation_fn=None, weights_initializer=tf.contrib.layers.xavier_initializer()) # Xavier Glorot and Yoshua Bengio (2010): Understanding the difficulty of training deep feedforward neural networks. 
#print(outputs) #Tensor("fully_connected/BiasAdd:0", shape=(?, 7), dtype=float32)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes]) #TODO Seq_length
#print(outputs) #Tensor("Reshape_2:0", shape=(1, 8, 7), dtype=float32)

### Seq2Seq Loss 계산 및 Train op
Time Step 마다 나오는 output과 거기에 대응되는 Target Y의 loss를 계산해줍니다.<br>
Tensorflow에서는 이를 위한 API인 tf.contrib.seq2seq.sequence_loss를 제공해줍니다.<br>
weights는 주로 padding이 있을때 masking 용도로 사용합니다.

In [11]:
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [12]:
prediction = tf.argmax(outputs, axis=2) # 0: Batch_size, 1: Sequence_length, 2: Num_classes

In [13]:
sess.run(tf.global_variables_initializer())

## Training & Evaluation
정의한 Operation을 session에 넣고 training을 시작합니다.<br>
모델이 예측하는 문자열의 순서를 확인합니다.<br>

In [14]:
for i in range(300):
        l, _ = sess.run([loss, train], feed_dict={X_input: x_input_one_hot, Y: y_data, seq_length:[8]}) # seq_length:[sequence_length] (Batch_size, Seq)
        result = sess.run(prediction, feed_dict={X_input: x_input_one_hot, seq_length:[8]})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [index2Char_ko[c] for c in np.squeeze(result)]
        print("RNN 모델의 예측 결과: ", ''.join(result_str))
print("End:)")

0 loss: 2.07264 prediction:  [[3 3 3 3 0 0 0 0]] true Y:  [[0, 1, 2, 3, 4, 5, 6, 7]]
RNN 모델의 예측 결과:  를를를를나나나나
1 loss: 2.06585 prediction:  [[3 3 3 3 0 0 0 0]] true Y:  [[0, 1, 2, 3, 4, 5, 6, 7]]
RNN 모델의 예측 결과:  를를를를나나나나
2 loss: 2.05838 prediction:  [[3 3 3 3 0 0 0 0]] true Y:  [[0, 1, 2, 3, 4, 5, 6, 7]]
RNN 모델의 예측 결과:  를를를를나나나나
3 loss: 2.05022 prediction:  [[3 3 3 3 0 0 0 0]] true Y:  [[0, 1, 2, 3, 4, 5, 6, 7]]
RNN 모델의 예측 결과:  를를를를나나나나
4 loss: 2.04139 prediction:  [[3 3 3 3 0 0 0 0]] true Y:  [[0, 1, 2, 3, 4, 5, 6, 7]]
RNN 모델의 예측 결과:  를를를를나나나나
5 loss: 2.0319 prediction:  [[3 3 3 3 0 0 0 0]] true Y:  [[0, 1, 2, 3, 4, 5, 6, 7]]
RNN 모델의 예측 결과:  를를를를나나나나
6 loss: 2.02178 prediction:  [[3 3 3 3 0 0 0 0]] true Y:  [[0, 1, 2, 3, 4, 5, 6, 7]]
RNN 모델의 예측 결과:  를를를를나나나나
7 loss: 2.01104 prediction:  [[3 3 3 3 0 0 0 5]] true Y:  [[0, 1, 2, 3, 4, 5, 6, 7]]
RNN 모델의 예측 결과:  를를를를나나나랑
8 loss: 1.99968 prediction:  [[3 3 3 3 0 0 0 5]] true Y:  [[0, 1, 2, 3, 4, 5, 6, 7]]
RNN 모델의 예측 결과:  를를를를나나나랑
9 loss: 1.9